# Tutorial: Removendo for-loops e tornando seu código `numpy` mais legível

## 1. Multiplicação de matrizes (e coisas semelhantes)

Digamos que queremos multiplicar matrizes que existem em alguma *grade de tamanho arbitrário*. \
A maneira mais intuitiva de fazer isso é criar um loop $\texttt{for}$ que percorra a grade:

In [ ]:
import numpy as np

H, W, i, j, k = 125, 133, 3, 4, 5 # Alguns números aleatórios

def slow_for_loop(A, B):
    C = np.zeros((H, W, i, k))
    for h in range(H):
        for w in range(W):
            C[h, w] = A[h, w] @ B[h, w]
    return C

In [ ]:
A = np.random.randn(H, W, i, j)
B = np.random.randn(H, W, j, k)

A seguir, veremos quanto tempo essa função leva para ser executada

In [ ]:
%timeit slow_for_loop(A, B)

Esse tempo não é um bom resultado, especialmente se isso tiver que ser feito muitas vezes em seu programa. \
Pior ainda, se uma das matrizes estiver ordenada incorretamente (por exemplo, se B tiver a forma (H, W, k, j)), a função acima falhará imediatamente. \
Mas não se preocupe! \
Em vez disso, podemos usar $\texttt{einsum}$!

In [ ]:
%timeit np.einsum('hwij, hwjk -> hwik', A, B)

Aqui, a string que fornecemos à função $\texttt{einsum}$ descreve exatamente a operação que aplicamos às entradas. \
Nesse caso, dizemos que A tem as dimensões (h, w, i, j) e B tem (h, w, j, k) e simplesmente dizemos que o resultado tem a dimensão (h, w, i, k). \
Isso é interpretado como uma multiplicação de A e B em que somamos todas as dimensões que não estão presentes no resultado. \
Como "j" não está presente na saída, sabemos que essa dimensão foi somada. \
Observe que isso é exatamente o mesmo que a multiplicação de matrizes em cada (h, w)!

## 2. Operações de Álgebra Linear mais avançadas
Felizmente, as operações mais avançadas são automaticamente agrupadas no numpy.
Vamos dar uma olhada em algumas!

### Determinante

In [ ]:
m, n = 3, 3
C = np.random.randn(H, W, m, n)
det = np.linalg.det(C)
print(det.shape)

### Inversa/Solução de sistemas de equações

In [ ]:
d = np.random.randn(H, W, n)
e = np.einsum('hwmn, hwn -> hwm', C, d)
Cinv = np.linalg.inv(C)
d_1 = np.einsum('hwnm, hwm -> hwn', Cinv, e)
d_2 = np.linalg.solve(C, e)

print(np.linalg.norm(d - d_1)) # Aprox. zero
print(np.linalg.norm(d - d_2)) # Aprox. zero

### Autovalores/Autovetores

In [ ]:
lam, V = np.linalg.eig(C)
print(lam.shape)
print(V.shape)

### Traço da Matriz

Por alguma razão, os autores do `numpy` decidiram que essa função deveria usar uma sintaxe completamente diferente.

In [ ]:
tr1 = np.trace(C, axis1=-2, axis2=-1)
print(tr1.shape)

# Também podemos usar a função einsum
tr2 = np.einsum('hwpp -> hw', C)
print(tr2.shape)
print(np.linalg.norm(tr1 - tr2))

## 3. Estudo de caso: Uma operação difícil

Digamos que temos uma matriz simétrica em cada ponto de uma grade. \
Queremos alterar essas matrizes. \
Em vez disso, queremos ter matrizes com os mesmos autovetores, mas cujos autovalores sejam os autovalores exponenciados e negados da matriz original. \
Fazer isso pode ser bastante complicado sem os loops $\texttt{for}$, mas vamos tentar!

In [ ]:
N = 10
J = np.random.randn(N, H, W, 2)
T = np.einsum('nhwd, nhwD -> hwdD', J, J) # Cria um tensor simétrico somando os produtos externos de alguns vetores
lam, V = np.linalg.eig(T)
lam = np.exp(-lam)
# Lembre-se de que T = V @ lam @ V^T, pois T é simétrico
T_2 = np.einsum('hwij, hwj, hwkj -> hwik', V, lam, V)

E isso é tudo o que precisamos fazer! \
Para recapitular o código acima. 
1. Criamos uma matriz simétrica em cada ponto somando alguns produtos externos de vetores.
2. Obtivemos a eigendecomposição em cada ponto
3. Exponenciamos os valores negados de nossa matriz
4. Reunimos a matriz novamente por multiplicação de matriz usando $\texttt{einsum}$

## 4. Notas finais

Analisamos algumas operações padronizadas e vetorizadas da Álgebra Linear em $\texttt{numpy}$ e mostramos que você pode fazer praticamente qualquer cálculo sem precisar recorrer a loops $\texttt{for}$ desagradáveis :)

Entretanto, aprender essa sintaxe leva tempo. \
Portanto, a recomendação é que você experimente vários desses tipos de operações e dê uma olhada em outras publicações de blogs sobre esse assunto:

Mais informações sobre o $\texttt{einsum}$: \
https://ajcr.net/Basic-guide-to-einsum/ \
e: https://rockt.github.io/2018/04/30/einsum